In [7]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import IntegerType, FloatType

# Step 1: Create a SparkSession
spark = SparkSession.builder \
    .appName("MovieRecommendations") \
    .master("local[*]") \
    .getOrCreate()

# Step 2: Create a DataFrame with sample user-movie ratings
data = [
    (0, 1, 4.0),  # User 0 rated Movie 1 with 4.0
    (0, 2, 2.0),  # User 0 rated Movie 2 with 2.0
    (1, 2, 5.0),  # User 1 rated Movie 2 with 5.0
    (1, 3, 3.0),  # User 1 rated Movie 3 with 3.0
    (2, 1, 1.0),  # User 2 rated Movie 1 with 1.0
    (2, 3, 4.0)   # User 2 rated Movie 3 with 4.0
]
columns = ["UserID", "MovieID", "Rating"]
ratings_df = spark.createDataFrame(data, columns)

# Step 3: Create an ALS model
als = ALS(
    maxIter=5,  # Number of iterations
    regParam=0.01,  # Regularization parameter
    userCol="UserID",  # Column for users
    itemCol="MovieID",  # Column for items
    ratingCol="Rating",  # Column for ratings
    coldStartStrategy="drop"  # Drop rows with null predictions
)

# Step 4: Train the ALS model
model = als.fit(ratings_df)

# Step 5: Generate top 2 movie recommendations for each user
user_recommendations = model.recommendForAllUsers(2)

# Step 6: Show the recommendations
user_recommendations.show(truncate=False)

# Stop the SparkSession
spark.stop()

+------+--------------------------------+
|UserID|recommendations                 |
+------+--------------------------------+
|0     |[{1, 3.9940298}, {2, 2.0000753}]|
|1     |[{2, 4.9965606}, {3, 3.0001903}]|
|2     |[{3, 3.9957175}, {2, 3.9366403}]|
+------+--------------------------------+

